# DynamoDB Learning
I've used DynamoDB before, but not in Java. This notebook contains examples of using the DynamoDB client from the AWS SDK V2 to interact with DyanmoDB tables

In [7]:
%maven com.amazonaws:aws-java-sdk-dynamodb:1.11.106

In [8]:
// Little class used for testing serializing and deseralizing to DynamoDB
class Folder {
    public String id; // DUFI
    public String ingestId; // Currently usually the DUFI
    public String metisProjectId;
    public int processingRecipeId;
    public String apid; // APID for the group
    public String dgs; // DGS number for the group
}

In [14]:
import com.amazonaws.services.dynamodbv2.AmazonDynamoDB;
import com.amazonaws.services.dynamodbv2.AmazonDynamoDBClientBuilder;
import com.amazonaws.services.dynamodbv2.document.DynamoDB;
import com.amazonaws.services.dynamodbv2.document.*;
import java.util.ArrayList;
import java.util.HashMap;

AmazonDynamoDB client = AmazonDynamoDBClientBuilder
    .standard()
    .build();
DynamoDB docClient = new DynamoDB(client);


void saveFolder(Folder folder) {
    Table table = docClient.getTable("dsw88-test-table");
    Item toSave = new Item()
        .withString( "MyPartitionKey", folder.id )
        .withString( "MySortKey", folder.apid )
        .withString( "IngestId", folder.ingestId )
        .withString( "MetisProjectId", folder.metisProjectId )
        .withNumber( "ProcessingRecipeId", folder.processingRecipeId );
    table.putItem(toSave);
}

Folder getFolder(String partitionKey, String sortKey) {
    Table table = docClient.getTable("dsw88-test-table");
    Item item = table.getItem(
        "MyPartitionKey",
        partitionKey,
        "MySortKey",
        sortKey
    );
    Folder folder = new Folder();
    folder.id = item.getString("MyPartitionKey");
    folder.apid = item.getString("MySortKey");
    folder.ingestId = item.getString("IngestId");
    folder.metisProjectId = item.getString("MetisProjectId");
    folder.processingRecipeId = item.getNumber("ProcessingRecipeId").intValue();
    return folder;
}

void getFolders(String partitionKey)
{
    Table table = docClient.getTable("dsw88-test-table");
    ItemCollection<QueryOutcome> items = table.query("MyPartitionKey", partitionKey);

    Iterator<Item> iterator = items.iterator();
    Item item = null;
    while (iterator.hasNext()) {
        item = iterator.next();
        System.out.println(item.toJSONPretty());
    }
}

In [15]:
// Add folder #1
Folder folder = new Folder();
folder.id = "SomeDUFI";
folder.apid = "SomeAPID";
folder.ingestId = "SomeDUFI";
folder.metisProjectId = "MMMM-MMM";
folder.processingRecipeId = 113;
saveFolder(folder);

// Add folder #2
Folder folder2 = new Folder();
folder2.id = "SomeDUFI";
folder2.apid = "SomeOtherAPID";
folder2.ingestId = "SomeOtherDUFI";
folder2.metisProjectId = "MMMM-MMM";
folder2.processingRecipeId = 113;
saveFolder(folder2);

Folder retFolder = getFolder("SomeDUFI", "SomeAPID");
System.out.println(retFolder.id);
System.out.println(retFolder.apid);
System.out.println(retFolder.ingestId);
System.out.println(retFolder.metisProjectId);
System.out.println(retFolder.processingRecipeId);

getFolders("SomeDUFI");

SomeDUFI
SomeAPID
SomeDUFI
MMMM-MMM
113
{
  "MyPartitionKey" : "SomeDUFI",
  "MetisProjectId" : "MMMM-MMM",
  "IngestId" : "SomeDUFI",
  "ProcessingRecipeId" : 113,
  "MySortKey" : "SomeAPID"
}
{
  "MyPartitionKey" : "SomeDUFI",
  "MetisProjectId" : "MMMM-MMM",
  "IngestId" : "SomeOtherDUFI",
  "ProcessingRecipeId" : 113,
  "MySortKey" : "SomeOtherAPID"
}
